In [4]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from collections import Counter
import seaborn
from ipywidgets import interact, Output
import ipywidgets as widgets

from charts_building import data_processing

###### These are configurations to be used for all plots

In [ ]:
 # will show up to 4
price_min = 5000
price_max = 500000  # this will cut off some offers, but will make charts more informative
meter_price_min = 50
meter_price_max = 4000
hist_resolution = 200  # all values will be divided in this many groups
area_min = 0
area_max = 1000

###### Loading all data into Pandas DataFrame

In [2]:
data = data_processing.read_data('houses', 'C:\Projects\l-site-parsing\offers.db')

In [3]:
data.head(2)

,price,total_area,rooms,floors_in_house,parsed_date,offer_title,offer_from,house_type,district,offer_added_date,text,offer_url,land_area
olx_id,,,,,,,,,,,,,
68835071,100000,400.0,6.0,3.0,2019-09-16,Продам дом,Частного лица,Дом,Индустриальный,2019-09-09,"Продам дом на ХТЗ.Участок 7 соток, 2-ва дома н...",https://www.olx.ua/obyavlenie/prodam-dom-ID4EP...,7.0
84257427,40000,120.0,7.0,1.0,2019-09-16,Продам отдельно стоящее здание,Частного лица,Дом,Московский,2019-09-10,Продам отдельно стоящее здание площадь 115 м2|...,https://www.olx.ua/obyavlenie/prodam-otdelno-s...,1.0


Data cleanup. Filtering out disctricts from outside the city. Removing offers with unreal values for price and area.

In [ ]:
price_square_meter = data.price / data.total_area
price_square_meter.head(2)

### Code for histogram

In [ ]:
def hist_set_x_axis(value):
    if value == 'Price':
        x_axis = data.price
        range_min = price_min
        range_max = price_max
    if value == 'Price per square meter':
        x_axis = price_square_meter
        range_min = meter_price_min
        range_max = meter_price_max
    if value == 'Total area':
        x_axis = data.total_area
        range_min = area_min
        range_max = area_max   
    if value == 'Land area':
        x_axis = data.land_area
        range_min = 0
        range_max = 20
        
    return (x_axis, range_min, range_max)

In [ ]:
hist_x_axis_dropdown = widgets.Dropdown(
    options=['Price', 'Price per square meter', 'Total area', 'Land area'], 
    value='Price')

In [ ]:
def build_histogram(x_axis):
    a, range_min, range_max = hist_set_x_axis(x_axis)
    plt.figure(figsize=(20,5), dpi=300, facecolor='white')
    plt.hist(a, hist_resolution, range=(range_min, range_max))
    plt.show()

In [ ]:
hist_out = Output()

In [ ]:
def x_axis_observer(bunch):
    hist_out.clear_output()
    with hist_out:
        build_histogram(x_axis = bunch.new)

In [ ]:
hist_x_axis_dropdown.observe(x_axis_observer, names='value')

In [ ]:
display(hist_x_axis_dropdown)
display(hist_out)

### Code for bar chart

In [ ]:
def bar_set_x_axis(value):
    rotation = 0
    if value == 'District':
        counter = Counter(data['district'])
    if value == 'Number of rooms':
        counter = Counter(data['rooms'])
    if value == 'House type':
        counter = Counter(data['house_type'])
        rotation = 30
    if value == 'Offer from':
        counter = Counter(data['offer_from'])
    if value == 'Floors in a house':
        counter = Counter(data['floors_in_house'])
        
    labels = dict(counter.most_common()).keys()
    height = dict(counter.most_common()).values()
    x_axis = np.arange(len(labels))
    
    return (x_axis, height, labels, rotation)

In [ ]:
bar_x_axis_dropdown = widgets.Dropdown(
    options=['District', 'Number of rooms', 'House type', 'Offer from', 'Floors in a house',], 
    value='District')

In [ ]:
def build_bar_chart(x_axis):
    x_axis, height, labels, rotation = bar_set_x_axis(x_axis)
    # Plot histogram using matplotlib bar().
    plt.figure(figsize=(20,5), dpi=300, facecolor='white')
    width = 0.9
    plt.bar(x_axis, height, width, align='center')
    plt.xticks(x_axis, labels, rotation=rotation)
    plt.show()

In [ ]:
bar_out = Output()

In [ ]:
def bar_x_axis_observer(bunch):
    bar_out.clear_output()
    with bar_out:
        build_bar_chart(x_axis = bunch.new)

In [ ]:
bar_x_axis_dropdown.observe(bar_x_axis_observer, names='value')

In [ ]:
display(bar_x_axis_dropdown)
display(bar_out)